In [2]:
import pandas as pd

g_data = pd.read_csv('/Users/joshh./Downloads/course_mapping_project/course_data_gpx/GCSATX_green_1/track_points.csv')
g_data.head()

,X,Y,track_fid,track_seg_id,track_seg_point_id,ele,time,magvar,geoidheight,name,...,link2_type,sym,type,fix,sat,hdop,vdop,pdop,ageofdgpsdata,dgpsid
0,-98.759960,29.388571,0,0,0,225.883,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-98.759973,29.388566,0,0,1,225.846,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-98.759978,29.388564,0,0,2,225.831,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-98.759984,29.388561,0,0,3,225.810,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-98.759988,29.388559,0,0,4,225.797,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# select columns as a list (not a tuple) to avoid KeyError
#g_data_point = g_data[['ele', 'X', 'Y']]

# save without adding an extra index column
#g_data_point.to_csv('/Users/joshh./Downloads/course_mapping_project/course_data_gpx/GCSATX_green_1/green_elevation_data2.csv', index=False)

In [31]:
import plotly.graph_objects as go

import pandas as pd

# Read data from a csv
y_data = pd.read_csv('/Users/joshh./Downloads/course_mapping_project/course_data_gpx/GCSATX_green_1/track_points.csv')

fig = go.Figure(data=[go.Surface(z=y_data.values)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True))
fig.update_layout(title=dict(text=''), autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=1000, height=1000,
                  margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

In [61]:
# don't re-import plotly.graph_objects here (already imported in another cell as `go`)
from plotly.subplots import make_subplots
from scipy.interpolate import griddata
from scipy.spatial import qhull
import numpy as np

# Use existing dataframe variable (g_data_point) already present in the notebook
data = g_data_point  # DataFrame with columns: ['ele', 'X', 'Y']
x = data['X'].values
y = data['Y'].values
z_vals = data['ele'].values * 3.28084  # Convert meters to feet

# Create a regular grid covering the data extent
n_grid = 50
xi = np.linspace(x.min(), x.max(), n_grid)
yi = np.linspace(y.min(), y.max(), n_grid)
X_grid, Y_grid = np.meshgrid(xi, yi)

# Robust interpolation: try cubic, fall back to linear or nearest if Qhull errors or nan regions occur
try:
    Z_grid = griddata((x, y), z_vals, (X_grid, Y_grid), method='cubic')
except qhull.QhullError:
    # Try linear with a small jitter to avoid degenerate input if necessary
    jitter = 1e-10 * max(np.ptp(x), np.ptp(y), 1.0)
    x_j = x + np.random.RandomState(0).normal(scale=jitter, size=x.shape)
    y_j = y + np.random.RandomState(1).normal(scale=jitter, size=y.shape)
    Z_grid = griddata((x_j, y_j), z_vals, (X_grid, Y_grid), method='linear')

# If cubic produced NaNs outside convex hull, fill those with nearest
if np.isnan(Z_grid).any():
    Z_nearest = griddata((x, y), z_vals, (X_grid, Y_grid), method='nearest')
    Z_grid = np.where(np.isnan(Z_grid), Z_nearest, Z_grid)

# For comparison, add the same surface with different colorscales to the other subplots
fig.add_trace(
    go.Surface(x=X_grid, y=Y_grid, z=Z_grid, colorscale='RdBu', showscale=True),
    #row=1, col=2
)

fig.update_layout(
    title_text='Interpolated elevation surface for CGSATX #',
    height=1500,
    width=1500
)

fig.show()